# Imports

In [22]:
# Imports für ersten Teil:
import requests
import pandas as pd
from tqdm import tqdm
import time

# Imports für zweiten Teil:
import requests
import json
import time
from tqdm import tqdm

# Erster Teil:
## Zitationen mithilfe der Metadaten von 'Dataset Builder' aus OpenAlex herausarbeiten

<br>
Ausgabedatei:
<br>
- zitationsdaten_hypatia_openalex_2017-2018.json mit Informationen zu Autoren, Titel und Referenzen (openAlex-URL) aller gefundenen Artikel aus der Zeitschrift Hypatia zwischen 2017 und 2018
<br>
- fehlerhafte_titel.txt: Alle nicht in OpenAlex gefundenen Zeitschriften aus diesem Zeitraum

In [2]:
# CSV mit Metadaten laden
df = pd.read_csv("./hypatia_metadaten_2017-2018.csv")
titel_liste = df['title'].dropna().tolist()

zitationsdaten = []
fehlerhafte_titel = []  # hier werden problematische Titel gespeichert

base_url = "https://api.openalex.org/works"

# Für jeden Titel in der Liste
for titel in tqdm(titel_liste, desc="🔍 Suche Artikel in OpenAlex"):
    try: # Suche Titel in OpenAlex
        params = {
            "filter": f"title.search:{titel}",
            "per-page": 1
        }
        response = requests.get(base_url, params=params) # OpenAlex Anfrage
        if response.status_code == 200: # Wenn Website richtig reagiert
            ergebnis = response.json() # Ergebnisanfrage in json formatieren
            if ergebnis.get("results"): # Wenn in ergebnis etwas ist
                paper = ergebnis["results"][0] # paper ist erstes Ergebnis in OpenAlex
                
                # Informationen für Suchergebnis in eintrag speichern
                eintrag = {
                    "OriginalTitle": titel,
                    "GefundenerTitel": paper.get("title"),
                    "Jahr": paper.get("publication_year"),
                    "Autoren": [a["author"]["display_name"] for a in paper.get("authorships", [])],
                    "Anzahl_Zitationen": paper.get("cited_by_count"),
                    "Anzahl_Referenzen": len(paper.get("referenced_works", [])),
                    "Referenzen": paper.get("referenced_works", []),
                    "DOI": paper.get("doi"),
                    "OpenAlex_ID": paper.get("id")
                }

                zitationsdaten.append(eintrag) # eintrag in zitationsdaten speichern
            else:
                print(f"❌ Kein Treffer für: {titel}")
                fehlerhafte_titel.append(titel)
        else:
            print(f"❌ Fehlerhafte Anfrage: {titel}")
            fehlerhafte_titel.append(titel)
        time.sleep(1)
    except Exception as e:
        print(f"⚠️ Ausnahme bei Titel: {titel}\n{e}")
        fehlerhafte_titel.append(titel)
        time.sleep(2)

# Ergebnis speichern
zitations_df = pd.DataFrame(zitationsdaten)
zitations_df.to_json("zitationsdaten_hypatia_openalex_2017-2018.json", orient="records", indent=2)

# Fehlerhafte Titel speichern
with open("fehlerhafte_titel.txt", "w", encoding="utf-8") as f:
    for t in fehlerhafte_titel:
        f.write(t + "\n")

print(f"\n✅ Fertig! {len(zitationsdaten)} Einträge erfolgreich gespeichert.") # Anzahl an erfolgreich gespeicherten Artikeln
print(f"⚠️ {len(fehlerhafte_titel)} Titel konnten nicht gefunden werden – siehe 'fehlerhafte_titel.txt'") # Anzahl an nicht gefundenen Artikeln



🔍 Suche Artikel in OpenAlex:   1%|▍                                                   | 1/132 [00:01<03:29,  1.60s/it]

❌ Fehlerhafte Anfrage: Philosophy's Folds: Seneca, Cavarero, and the History of Rectitude


🔍 Suche Artikel in OpenAlex:   2%|▊                                                   | 2/132 [00:03<03:26,  1.59s/it]

❌ Fehlerhafte Anfrage: The "Bio" Politics of Matter and Mattering for Feminist Engagements with Biosocial, Biocultural, and Posthuman Embodiment


🔍 Suche Artikel in OpenAlex:   8%|████▎                                              | 11/132 [00:17<03:05,  1.54s/it]

...

❌ Fehlerhafte Anfrage: "Everything Being Tangled Up in Every Other Thing": Class, Desire, and Shame in Michelle Tea's "The Passionate Mistakes and Intricate Corruption of One Girl in America"


🔍 Suche Artikel in OpenAlex:  99%|█████████████████████████████████████████████████▌| 131/132 [03:24<00:01,  1.53s/it]

❌ Fehlerhafte Anfrage: The Role of Darwin in Elizabeth Grosz's Deleuzian Feminist Theory: Sexual Difference, Ontology, and Intervention


🔍 Suche Artikel in OpenAlex: 100%|██████████████████████████████████████████████████| 132/132 [03:25<00:00,  1.56s/it]


✅ Fertig! 103 Einträge erfolgreich gespeichert.
⚠️ 29 Titel konnten nicht gefunden werden – siehe 'fehlerhafte_titel.txt'


# Zweiter Teil
## Namen der Referenzen herausarbeiten
(Läuft wegen der Menge an Referenzen etwas länger.)

<br>
Ausgabedatei:
- referenz_autoren.json: Alle in Hypatia referenzierten Titel mit Informationen zu OpenAlex-URL, Titel, Jahr und AutorInnen

In [5]:
# Lade die Liste deiner Referenz-URLs oder -IDs
with open("zitationsdaten_hypatia_openalex_2017-2018.json", "r", encoding="utf-8") as f:
    daten = json.load(f)

# Alle eindeutigen Referenz-IDs sammeln
referenz_ids= set()
for eintrag in daten:
    referenz_ids.update(eintrag.get("Referenzen", []))

# Datenstruktur für Ergebnisse
referenz_infos = {}

# Nach drei Versuchen zum nächsten skippen
MAX_RETRIES = 3

# Hauptloop über alle Referenz-IDs
for rid in tqdm(referenz_ids, desc="Hole Referenzdaten von OpenAlex"):
    # Extrahiere die ID z. B. "W1234567890" aus URL
    if rid.startswith("https://openalex.org/"):
        work_id = rid.split("/")[-1]
    else:
        work_id = rid

    # Baue API-URL korrekt
    api_url = f"https://api.openalex.org/works/{work_id}"

    for attempt in range(MAX_RETRIES):
        try:
            response = requests.get(api_url)

            # Erfolgreiche und nicht-leere Antwort
            if response.status_code == 200 and response.content.strip():
                paper = response.json()
                autoren = [a["author"]["display_name"] for a in paper.get("authorships", [])]

                referenz_infos[rid] = {
                    "title": paper.get("title"),
                    "authors": autoren,
                    "year": paper.get("publication_year")
                }
                break  # fertig, weiter zur nächsten ID
            else:
                time.sleep(2)

        except Exception as e:
            if attempt == MAX_RETRIES - 1:
                referenz_infos[rid] = {"error": str(e)}
            time.sleep(2)

# Speichere alle Daten
with open("referenz_autoren.json", "w", encoding="utf-8") as f:
    json.dump(referenz_infos, f, indent=2, ensure_ascii=False)

print("✅ Referenzdaten wurden gespeichert in 'referenz_autoren.json'")


Hole Referenzdaten von OpenAlex: 100%|█████████████████████████████████████████████| 2006/2006 [20:15<00:00,  1.65it/s]

✅ Referenzdaten wurden gespeichert in 'referenz_autoren.json'


### Ergebnisstruktur der Daten in 'referenz-autoren.json': 

"https://openalex.org/W2052757464": {
    "title": "The Moral Sex: Woman's Nature in the French Enlightenment",
    "authors": [
      "Londa Schiebinger",
      "Lieselotte Steinbrügge",
      "Pamela E. Selwyn"
    ],
    "year": 1995
  },